In [9]:
"""
Deep Neural Networks on Not MNIST using Keras
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments
"""
# On command line: python3 mnist_a2j_mlp_keras.py
# Prerequisite: tensorflow 1.0 and keras 2.0
# must run mnist_a2j_2pickle.py first (one-time) to generate the data

from __future__ import print_function

import numpy as np
import pickle
import time

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

## use of pickle to speed up loading of data
# pickle_file = open( "mnist_a2j.pickle", "rb" )
# data = pickle.load(pickle_file)
# test_labels = data["test_labels"]
# train_labels = data["all_labels"]
# test_dataset = data["test_dataset"]
# train_dataset = data["all_dataset"]
# del data
# pickle_file.close()

# print("Training size: ", train_dataset.shape)
# print("Training labels: ", train_labels.shape)
# print("Test size: ", test_dataset.shape)
# print("Test labels: ", test_labels.shape)

Using TensorFlow backend.


In [4]:
from get_log_data import read_logs, get_log_traintest,get_digitized_logdata, get_labelized_logdata
x_train, x_test, y_train, y_test, scaler_x, scaler_y=get_log_traintest('vwcl',True,False)

In [5]:
min(y_train),max(y_train),min(y_test),max(y_test)

(array([0.]), array([1.]), array([0.]), array([1.]))

In [6]:
y_test

array([[0.7401],
       [0.7354],
       [0.7529],
       ...,
       [0.6602],
       [0.6635],
       [0.6871]])

In [13]:
from get_log_data import read_logs, get_log_traintest,get_digitized_logdata, get_labelized_logdata
num_labels =2# train_labels.shape[1]
barrier=0.5
train_dataset, test_dataset, train_labels, test_labels=get_labelized_logdata(num_labels,ref=barrier,LABEL='vwcl') #LABEL='phit'

image_size = 28
input_size = 7 #image_size*image_size
batch_size = 1028
hidden_units = 1024
learning_rate = 0.5
dropout = 0.8

model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size))
model.add(Activation('relu'))

model.add(Dropout(dropout))
model.add(Dense(hidden_units))
model.add(Activation('relu'))

model.add(Dropout(dropout))
model.add(Dense(num_labels))
model.add(Activation('softmax'))

sgd = SGD(lr=learning_rate) # , decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(train_dataset, train_labels,
          epochs=75,
          batch_size=batch_size, shuffle=False)
score = np.asarray(model.evaluate(test_dataset, test_labels, batch_size=batch_size))*100.0
# Accuracy: 86.0%
print("\nTest accuracy: %.1f%%" % score[1])
print("Elapsed: " , elapsed(time.time() - start_time))

Epoch 1/75
14853/14853 [==============================] - 1s 69us/step - loss: 0.3636 - acc: 0.8453
Epoch 2/75
14853/14853 [==============================] - 1s 38us/step - loss: 0.1533 - acc: 0.9346
Epoch 3/75
14853/14853 [==============================] - 1s 41us/step - loss: 0.1531 - acc: 0.9329
Epoch 4/75
14853/14853 [==============================] - 1s 36us/step - loss: 0.1446 - acc: 0.9366
Epoch 5/75
14853/14853 [==============================] - 1s 36us/step - loss: 0.1320 - acc: 0.9416
Epoch 6/75
14853/14853 [==============================] - 1s 37us/step - loss: 0.1260 - acc: 0.9450: 0s - loss: 0.1827 - acc: 
Epoch 7/75
14853/14853 [==============================] - 1s 38us/step - loss: 0.1232 - acc: 0.9463
Epoch 8/75
14853/14853 [==============================] - 1s 40us/step - loss: 0.1168 - acc: 0.9502
Epoch 9/75
14853/14853 [==============================] - 1s 39us/step - loss: 0.1149 - acc: 0.9506
Epoch 10/75
14853/14853 [==============================] - 1s 38us/step -

In [14]:
model.predict(test_dataset),test_labels
test_pred=np.array([0 if b<0.5 else 1 for a,b in model.predict(test_dataset)])
print(sum(test_labels-test_pred))
count=0
neg_count=0
for a, b in zip(test_labels,test_pred):
    if a==b:
        count +=1
        
    else:
        neg_count +=1
count,neg_count,count+neg_count,count/(count+neg_count)

-95.0


(1784, 103, 1887, 0.9454160042395336)

In [15]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(test_labels,test_pred))
print(classification_report(test_labels,test_pred))

[[ 168   99]
 [   4 1616]]
             precision    recall  f1-score   support

        0.0       0.98      0.63      0.77       267
        1.0       0.94      1.00      0.97      1620

avg / total       0.95      0.95      0.94      1887

